Che cos’è – Una covarianza tra il dosaggio allelico di un individuo moderno e la frequenza di un’ancestry antica, centrata rispetto alle medie della popolazione moderna e dell’intero insieme di ancestries.

Cosa misura – La “somiglianza locale” con una componente ancestrale in regioni funzionali (TAGR).

Perché è utile – Evita di usare effetti GWAS‐specifici (spesso distorti da struttura), controlla esplicitamente il confine fra genetica e geografia, individua divergenze di valore poligenico tra ancestries.

Perché le simulazioni SLiM – Servono solo a calibrare quanta covA ci si aspetta sotto deriva/sele­zione nota e a stimare potenza/falsi positivi; non sono obbligatorie per l’analisi empirica.


DATI:

**Genotipi antichi (AADR)** - v62.0_1240k_public.{geno,snp,ind} - Harward sito dell'articolo


**Genotipi “moderni”** da UK Biobank - https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ -  VCF files containing the variants detected and additional genotype VCF files listing genotypes for each individual at each variant location (provided per chromosome due to file size)

**NHGRI-EBI GWAS Catalog** https://www.ebi.ac.uk/gwas/

# Scariciamo plink2 (anche a parte serve per convertire i documenti modern dopo)
!wget -q https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip
!unzip -q plink2_linux_x86_64_latest.zip
!chmod +x plink2
!mv plink2 /usr/local/bin/
!plink2 --version

#!plink2 --vcf "tuo_file".vcf.gz --make-bed --out output_prefix

In [ ]:
# pip install numpy pandas cyvcf2 scikit-allel statsmodels tqdm

In [1]:
# Definizione dei gruppi ancestrali 

import pandas as pd, re, pathlib, itertools, subprocess, numpy as np
AADR_IND = pathlib.Path('/home/viannibe/codon_bias/CoA/data/antichi/v62.0_1240k_public.ind')

anc_map = {'WHG': ['Loschbour', 'Bichon', 'Villabruna'],
           'EEF': ['Bar31', 'Stuttgart', 'Iceman'],
           'SBA': ['Yamnaya', 'Afanasievo', 'Poltavka']}

for anc, patterns in anc_map.items():
    with open(f'data/{anc.lower()}.ids','w') as out:
        for line in open(AADR_IND):
            if any(re.search(p,line, re.I) for p in patterns):
                out.write(line.split()[0]+'\n')

# qui usiamo un “lite set” di ID; per accuratezza sfrutta la lista ufficiale nel paper)


In [3]:
# Costruzione delle TAGR

gwas = pd.read_csv('/home/viannibe/codon_bias/CoA/data/gws_catalog/gwas_catalog_v1.0.2-associations_e113_r2025-04-28(1).tsv', sep='\t', low_memory=False)

trait = 'standing height'
hits  = gwas[gwas['MAPPED_TRAIT'].str.contains(trait, case=False, na=False)]
bed   = hits[['CHR_ID','CHR_POS']].dropna()
bed['start'] = bed['CHR_POS']-10000
bed['end']   = bed['CHR_POS']+10000
bed[['CHR_ID','start','end']].to_csv(f'data/TAGR_{trait.replace(" ","_")}.bed',
                                     sep='\t', header=False, index=False)



In [ ]:
# Parse VCF moderni + Eigenstrat antichi

import scikit_allel as allel, cyvcf2, gzip, textwrap, os
from tqdm.auto import tqdm

# Carica moderni (dosaggi 0/1/2) limitandosi alle regioni TAGR
region_list = ['%s:%d-%d'%(c,s,e) for c,s,e in bed[['CHR_ID','start','end']].values]
vcf = cyvcf2.VCF("data/ALL.chr22*.vcf.gz")
mod_samples = vcf.samples
dosage = [] ; snps = []
for reg in tqdm(region_list):
    for rec in vcf(reg):
        snps.append((rec.CHROM, rec.POS))
        dosage.append(rec.genotypes)  # list of (a1,a2,phased)

G_mod = np.array([[a+b for a,b,_ in row] for row in dosage], dtype='int8').T  # shape (n_samples,n_snps)
Gm    = G_mod.mean(axis=0)


In [ ]:
# gli antichi bisogna convertirli prima con plink2

# plink2 --geno v62.0_1240k_public.geno \
#        --snp  v62.0_1240k_public.snp  \
#        --ind  v62.0_1240k_public.ind  \
#        --make-bed --out data/aadr

In [ ]:
anc = allel.read_plink('data/aadr.bed', fam_fields=['sample'])
anc_samps = anc['samples']

# media per SNP di ciascun ancestry
Gp = {}
for anc_key in ['WHG','EEF','SBA']:
    keep = np.isin(anc_samps, open(f'data/{anc_key.lower()}.ids').read().splitlines())
    Gp[anc_key] = anc['calldata/GT'].take(np.where(keep)[0], axis=1).to_n_alt().mean(axis=1)


In [ ]:
# Calcolo CovA

Xi = G_mod - Gm          # centering moderni
covA = {}
for k in Gp:
    Xp = Gp[k] - Gm
    covA[k] = Xi.dot(Xp)/Xp.size

covA_df = pd.DataFrame(covA, index=mod_samples)


In [ ]:
# Regressione sul fenotipo

import statsmodels.formula.api as smf

pheno = pd.read_csv('data/height_pheno.csv')      # sample,value
covar = pd.read_csv('data/covar.csv')

df = pheno.merge(covar).merge(covA_df, left_on='sample', right_index=True).dropna()

formula = 'value ~ WHG + EEF + SBA + age + sex + PC1 + PC2 + center + TDI'
res = smf.ols(formula, data=df).fit()
print(res.summary())


# (per un modello “alla Marnetto” togli due covA globali e limita WHG/EEF/SBA a una per volta → tre modelli separati)


Coefficiente βcovA → interpreti in termini di contributo ancestrale al trait.

Se |β| > 0.03 e p < 0.05 FDR → possibile divergenza reale.

Confronta con i valori di riferimento del paper (Fig. 2A).

In [ ]:
res.save('results/height_covA.pkl')